### Mission: Code my own SoftMax Regression without Scikit-Learn Classifier

In [454]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

iris = datasets.load_iris()

X, y = iris.data, iris.target

# Preparing X
X_scaled = StandardScaler().fit_transform(X)
X_scaled = np.c_[np.ones((len(X_scaled), 1)) ,X_scaled] # 1 on left

y_encoded = OneHotEncoder().fit_transform(y.reshape(-1, 1)).toarray()

# Splitting Train / Val
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Training Vars
m, n = X_train.shape
k = len(np.unique(y))

# Initial Theta (shape must be nxk)
np.random.seed(50)
theta_initial = np.random.rand(n, k)

print('Training Info:')
print('m:', m)
print('n:', n, '(including theta0)')
print('k:', k)
print('X_train shape: ', X_train.shape)
print('Y_train shape: ', y_train.shape)
print('X_val shape: ', X_val.shape)
print('Y_val shape: ', y_val.shape)
print('Theta shape: ', theta.shape)

Training Info:
m: 120
n: 5 (including theta0)
k: 3
X_train shape:  (120, 5)
Y_train shape:  (120, 3)
X_val shape:  (30, 5)
Y_val shape:  (30, 3)
Theta shape:  (5, 3)


In [455]:
def calc_accuracy_score(y, y_train):
    y_label = np.argmax(y, axis=1)

    ok_preds = y_label == y_train
    return sum(ok_preds) / len(ok_preds)


def calc_accuracy(X, y, theta):
    """Calculates Model Accuracy"""
    y_targets = np.argmax(y, axis=1)

    y_preds = X.dot(theta)
    y_targets_preds = np.argmax(y_preds, axis=1)
    
    ok_preds = y_targets == y_targets_preds
    return round(sum(ok_preds) / len(ok_preds), 2)

def predict(X, theta):
    """Predicts some X"""
    pred = X.dot(theta)
    return np.argmax(pred, axis=1)

def softmax_function(pred_i_k, preds_i_k):
    """SoftMax Function
        pred_i_k: the score of the observation i for class k
        preds_i_k: the scores of the observatio i for all classes
    """
    return np.exp(pred_i_k) / (np.exp(preds_i_k).sum())

def cost_function(X, y, theta):
    """Returns the cost function"""
    sum_acc = 0
    
    m = len(X)
    _, K = theta.shape
    
    for i in range(0, m):
        preds_i = X[i].dot(theta) # Returns an array k dimensional, with the probability of each
        for k in range(0, K):
            yk = y[i][k]
            pk = softmax_function(preds_i[k], preds_i)
            
            sum_acc = sum_acc + yk * np.log(pk)
    return -1 / m * sum_acc

def calc_gradients(X, y, theta):
    """Return the gradients."""
    m = len(X)
    _, K = theta.shape
    
    grads = np.zeros_like(theta)
    
    for i in range(0, m):
        preds_i = X[i].dot(theta)
        for k in range(0, K):
            yk = y[i][k]
            pk = softmax_function(preds_i[k], preds_i)
            
            grads[:, k] = grads[:, k] + (pk - yk) * X[i]
    
    return 1 / m * grads
            

calc_gradients(X_train, y_train, theta)

array([[ 0.00031746, -0.0099579 ,  0.00964044],
       [ 0.00405785, -0.00299366, -0.0010642 ],
       [-0.00387146, -0.00123082,  0.00510228],
       [ 0.00752029,  0.00297834, -0.01049863],
       [ 0.00690526,  0.00387651, -0.01078177]])

In [456]:
def train_softmax_early_stopping(X_train, y_train, X_val, y_val,theta, epoch=0, train_errors=[], val_errors=[], eta=0.1, max_iters=1000):
#     print(f'***** Epoch: {epoch} *****')
#     print('Cost function:', cost_function(X_train, y_train, theta), '\n')

    train_accuracy = calc_accuracy(X_train, y_train, theta)
    val_accuracy = calc_accuracy(X_val, y_val, theta)
    
    train_errors.append(1 - train_accuracy)
    val_errors.append(1 - val_accuracy)
    
    gradients = calc_gradients(X_train, y_train, theta)
    theta = theta - eta * gradients
    
    if epoch > max_iters:
        return theta, train_errors, val_errors

    return train_softmax_early_stopping(X_train, y_train, X_val, y_val, theta, epoch + 1, train_errors, val_errors)


theta, train_errors, val_errors = train_softmax_early_stopping(X_train, y_train, X_val, y_val, theta_initial)

In [457]:
print(calc_accuracy(X_train, y_train, theta))
print(calc_accuracy(X_val, y_val, theta))

0.97
1.0
